In [ ]:
!pip install virtualenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 18.3 MB/s eta 0:00:00


In [ ]:
!virtualenv /content/drive/MyDrive/cuda_env

created virtual environment CPython3.10.12.final.0-64 in 853ms
  creator CPython3Posix(dest=/content/drive/MyDrive/cuda_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.3.1, setuptools==68.2.2, wheel==0.41.3
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
!source /content/drive/MyDrive/cuda_env/bin/activate;

In [ ]:

!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!nvidia-smi

Fri Nov 24 05:14:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-_wjlftwu
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-_wjlftwu
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4294 sha256=b2e8282f15b319d9dc6854e625914b74514bc31abf18e38689b3ae3c9187a1a3
  Stored in directory: /tmp/pip-ephem-wheel-cache-ka17gksy/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


**Task1 (Vector Element Square Root)**

In [ ]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include<cmath>

#define N 10

// Kernel function to perform vector multiplication
__global__ void vectorSqrt(float *a, float *result) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;

    if (tid < N) {
        result[tid] = sqrt(a[tid]);
    }
}

int main() {
    float h_a[N], h_result[N];

    // Device vectors
    float *d_a, *d_result;

    // Allocate memory on the device
    cudaMalloc((void **)&d_a, N * sizeof(float));
    cudaMalloc((void **)&d_result, N * sizeof(float));

    // Initialize host vectors with random values
    for (int i = 0; i < N; ++i) {
        h_a[i] = float((i+1)*10);  // Adjust the range as needed
        //printf("val = %f\n",h_a[i]);
    }
    //printf("\n");

    printf("Real vector:\n");
    for (int i = 0; i < N; ++i) {
        printf("%f ", h_a[i]);
    }
    printf("\n");

    // Copy host vectors to device
    cudaMemcpy(d_a, h_a, N * sizeof(float), cudaMemcpyHostToDevice);

    // Set up grid and block dimensions
    dim3 blockDim(256);
    dim3 gridDim((N + blockDim.x - 1) / blockDim.x);

    // Launch kernel
    vectorSqrt<<<gridDim, blockDim>>>(d_a, d_result);

    // Copy result back to host
    cudaMemcpy(h_result, d_result, N * sizeof(float), cudaMemcpyDeviceToHost);

    // Clean up
    cudaFree(d_a);
    cudaFree(d_result);



    // Display the result
    printf("Resultant vector:\n");
    for (int i = 0; i < N; ++i) {
        printf("%f ", h_result[i]);
    }
    printf("\n");

    return 0;
}


Real vector:
10.000000 20.000000 30.000000 40.000000 50.000000 60.000000 70.000000 80.000000 90.000000 100.000000 
Resultant vector:
3.162278 4.472136 5.477226 6.324555 7.071068 7.745967 8.366600 8.944272 9.486833 10.000000 



**Task2 Matrix Addition**

In [ ]:
%%cu

#include <iostream>

#include <cmath>

#include <cuda_runtime.h>

const int N = 4; // Size of the matrices (N x N)
// CUDA kernel to add two matrices
__global__ void matrixAdd(int * a, int * b, int * c, int n) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int j = blockIdx.y * blockDim.y + threadIdx.y;
  if (i < n && j < n) {
    int index = i * n + j;
    c[index] = a[index] + b[index];
  }
}
int main() {
  int * h_a, * h_b, * h_c; // Host matrices
  int * d_a, * d_b, * d_c; // Device matrices
  // Allocate memory on the host
  h_a = new int[N * N];
  h_b = new int[N * N];
  h_c = new int[N * N];
  // Initialize matrices on the host
  for (int i = 0; i < N * N; ++i) {
    h_a[i] = i;
    h_b[i] = 2 * i;
  }
  // Allocate memory on the device
  cudaMalloc((void ** ) & d_a, N * N * sizeof(int));
  cudaMalloc((void ** ) & d_b, N * N * sizeof(int));
  cudaMalloc((void ** ) & d_c, N * N * sizeof(int));
  // Copy matrices from host to device
  cudaMemcpy(d_a, h_a, N * N * sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, h_b, N * N * sizeof(int), cudaMemcpyHostToDevice);
  // Define block and grid sizes
  dim3 blockSize(16, 16);
  dim3 gridSize(ceil(static_cast < float > (N) / blockSize.x), ceil(static_cast < float > (N) / blockSize.y));
  // Launch the kernel
  matrixAdd << < gridSize, blockSize >>> (d_a, d_b, d_c, N);
  // Copy the result from device to host
  cudaMemcpy(h_c, d_c, N * N * sizeof(int), cudaMemcpyDeviceToHost);

  // Print the result
  std::cout << "Matrix A:\n";
  for (int i = 0; i < N; ++i) {
    for (int j = 0; j < N; ++j) {
      std::cout << h_a[i * N + j] << " ";
    }
    std::cout << "\n";
  }
  std::cout << "\nMatrix B:\n";
  for (int i = 0; i < N; ++i) {
    for (int j = 0; j < N; ++j) {
      std::cout << h_b[i * N + j] << " ";
    }
    std::cout << "\n";
  }
  std::cout << "\nResultant Matrix C:\n";
  for (int i = 0; i < N; ++i) {
    for (int j = 0; j < N; ++j) {
      std::cout << h_c[i * N + j] << " ";
    }
    std::cout << "\n";
  }
  // Free memory on the device
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);
  // Free memory on the host
  delete[] h_a;
  delete[] h_b;
  delete[] h_c;
  return 0;
}

Matrix A:
0 1 2 3 
4 5 6 7 
8 9 10 11 
12 13 14 15 

Matrix B:
0 2 4 6 
8 10 12 14 
16 18 20 22 
24 26 28 30 

Resultant Matrix C:
0 3 6 9 
12 15 18 21 
24 27 30 33 
36 39 42 45 

